# Load Data

First, some technicalities 

In [ ]:
import pandas as pd
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri

In [ ]:
# Enable pandas conversion for R data frames
pandas2ri.activate()

#### Source my R functions that handle Seurat objects

In [17]:
try:
    # Source the R script
    robjects.r.source("../scripts/data_loader.R")
    print("R script sourced successfully!")
except Exception as e:
    print("Error sourcing R script:", e)

R script sourced successfully!


In [35]:
load_seurat = robjects.globalenv["load_seurat"]
extract_raw_counts = robjects.globalenv["extract_raw_counts"]
load_cell_types = robjects.globalenv["load_cell_types"]

#### Load Seurat file (not public data) with R function

In [ ]:
%%capture

seurat_obj = load_seurat("../data/raw/SeuratProject.h5Seurat")

Load raw RNA-seq counts table (and transform to pandas)

In [ ]:
raw_counts_r = extract_raw_counts(seurat_obj)
# Convert the R data frame to a pandas DataFrame
raw_counts = pandas2ri.rpy2py_dataframe(raw_counts_r)
# print(raw_counts.head())

Load cell type annotations for all cells in raw counts

In [ ]:
cell_types_r = load_cell_types(seurat_obj)
cell_types = pandas2ri.rpy2py_dataframe(cell_types_r)
# print(cell_types.head())

In [41]:
raw_counts

,a,abd-A,Abd-B,Abl,abo,ac,acj6,Acph-1,Act5C,Act42A,...,lncRNA:CR43716,lncRNA:CR44997,asRNA:CR45151,lncRNA:CR45310,lncRNA:CR45425,asRNA:CR45822,asRNA:CR45891,lncRNA:CR45961,lncRNA:CR46032,lncRNA:CR46119
TP1_AACTCAGGTAAATACG,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,25.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TP1_TCACGAATCTATCGCC,0.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,27.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TP1_TACGGTACAATAGAGT,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TP1_CAACTAGAGAGACGAA,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,20.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TP1_GCGCCAAAGTCGATAA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TP2.2_AACACGTCATTCACTT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TP2.2_TCTTTCCGTACCGTTA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TP2.2_GATTCAGCACACGCTG,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,4.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TP2.2_TGAGCATGTGATGTCT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Save transformed data as CSV for later use 

In [43]:
raw_counts.to_csv("../data/raw_counts.csv")
cell_types.to_csv("../data/cell_types_annotations.csv")